In [ ]:
# base
import os
import shutil
import random
import numpy as np
import time
from collections import Counter
from sklearn.metrics import mean_squared_error
import pandas.util.testing as tm

# data preprocessing
import pandas as pd
pd.set_option('max_columns', 130, 'max_rows', 30)

# visualization
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
plt.rc('font', family='Malgun Gothic')
plt.rc('axes', unicode_minus=False)

# ignore warining
import warnings
warnings.filterwarnings('ignore')



In [ ]:
pip install selenium

In [ ]:
# 지도, 거리 관련
import folium # 지도 관련 시각화
from folium.plugins import MarkerCluster #지도 관련 시각화
import geopy.distance #거리 계산해주는 패키지 사용

# save
from sklearn.externals import joblib 
import pickle

# selenium
from selenium.webdriver import Chrome
import geopy.distance

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.externals import joblib 
import pickle

import random
random.seed(42)

import lightgbm as lgb
from keras import metrics
from sklearn.metrics import r2_score

In [ ]:
import os
print(os.getcwd())
!ls

# 실행시 등장하는 URL을 클릭하여 허용해주면 인증KEY가 나타난다. 복사하여 URL아래 빈칸에 붙여넣으면 마운트에 성공하게된다.
from google.colab import drive

In [ ]:
from google.colab import drive
drive.mount('/MyDrive')

In [ ]:
cd /MyDrive

In [ ]:
pd.options.display.max_rows = 3000
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [ ]:
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
import numpy as np
from sklearn.model_selection import train_test_split


<변수 선택법 - 보루타>

In [ ]:
"""
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier

!pip install boruta
from boruta import BorutaPy

# load X and y
# NOTE BorutaPy accepts numpy arrays only, hence the .values attribute
#X = pd.read_csv('examples/test_X.csv', index_col=0).values
#y = pd.read_csv('examples/test_y.csv', header=None, index_col=0).values
#y = y.ravel()
X = trainDeleteLabel.copy().values
y = y_train.copy().values
y = y.ravel()

# define random forest classifier, with utilising all cores and
# sampling in proportion to y labels
rf = RandomForestClassifier(n_jobs=-1, class_weight='balanced', max_depth=5)

# define Boruta feature selection method
feat_selector = BorutaPy(rf, n_estimators='auto', verbose=2, random_state=1)

# find all relevant features - 5 features should be selected
feat_selector.fit(X, y)

# check selected features - first 5 features are selected
feat_selector.support_

# check ranking of features
feat_selector.ranking_

# call transform() on X to filter it down to selected features
X_filtered = feat_selector.transform(X)
"""

In [ ]:
#indexes = np.where(feat_selector.support_ == True)

<모델링>

In [ ]:
modelTrain = pd.read_csv("/MyDrive/My Drive/TCL/modelTrain.csv")
modelTest = pd.read_csv("/MyDrive/My Drive/TCL/modelTest.csv")
y_train = pd.read_csv("/MyDrive/My Drive/TCL/allTrain_y.csv")
y_test= pd.read_csv("/MyDrive/My Drive/TCL/allTest_y.csv")

In [ ]:
modelTrain.drop(modelTrain.iloc[:,[0]], axis='columns', inplace=True)
modelTest.drop(modelTest.iloc[:,[0]], axis='columns', inplace=True)
y_train.drop(y_train.iloc[:,[0]], axis='columns', inplace=True)
y_test.drop(y_test.iloc[:,[0]], axis='columns', inplace=True)

(랜덤포레스트)


In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
params = { 'n_estimators' : [300, 500],
           'max_features' : [3,5],
           'min_samples_leaf' : [1,2],
           'min_samples_split' : [2]
}

rf_clf = RandomForestClassifier(random_state = 0, n_jobs = 1)
grid_cv = GridSearchCV(rf_clf, param_grid = params, cv = 3, n_jobs = 1)

In [ ]:
grid_cv.fit(modelTrain, y_train)

print('최적 하이퍼 파라미터: ', grid_cv.best_params_)
print('최고 예측 정확도: {:.4f}'.format(grid_cv.best_score_))

In [ ]:
rf = RandomForestRegressor(max_features=3,
                           min_samples_leaf=2,
                           min_samples_split=2,
                           n_estimators=500,
                           random_state=1217)

X = modelTrain
y = y_train

In [ ]:
rf.fit(X, y)

In [ ]:
y_pred = rf.predict(modelTrain)

In [ ]:
y_pred = rf.predict(modelTest)

In [ ]:
np.sqrt(mean_squared_error(y_test, y_pred)**0.5) # RMSE = 1.4682414153742127

( LGBM )

In [ ]:
lgbm = lgb.LGBMRegressor(num_iterations = 1000, 
                                learning_rate = 0.05,
                                boosting = 'dart',
                         Metric = 'regression_l2', n_jobs=1)

X = modelTrain
y = y_train

In [ ]:
X.columns=range(len(X.columns))

In [ ]:
lgbm.fit(X, y)

In [ ]:
y_pred = lgbm.predict(X)

In [ ]:
np.sqrt(mean_squared_error(y_train, y_pred)**0.5)

In [ ]:
y_pred = lgbm.predict(modelTest)

In [ ]:
np.sqrt(mean_squared_error(y_train, y_pred)**0.5)

In [ ]:
<k-fold 검정>

In [ ]:
from sklearn.metrics import make_scorer, mean_squared_error
from sklearn.model_selection import cross_val_score
scorer = make_scorer(mean_squared_error, False)

clf = RandomForestRegressor(n_estimators=500, n_jobs=-1)
cv_score = np.sqrt(-cross_val_score(estimator=clf, X=modelTrain, y= y_train, cv=15, scoring = scorer))

plt.figure(figsize=(10,5))
plt.bar(range(len(cv_score)), cv_score)
plt.title('Cross Validation Score')
plt.ylabel('RMSE')
plt.xlabel('Iteration')

plt.plot(range(len(cv_sore) +1), [cv_score.mean()] * (len(cv_score) +1 ))
plt.tight_layout()

In [ ]:
(베이스라인 모델 - regression)

In [ ]:
from sklearn.linear_model import LinearRegression
mlr = LinearRegression()
mlr.fit(X_trainLm, y_trainLm) 

In [ ]:
y_predicted = mlr.predict(X_testLm)

In [ ]:
np.sqrt(mean_squared_error(y_train, y_predicted)**0.5)

<lgbm>